In [172]:
import pandas as pd

import progressbar

import urllib

In [173]:
bar = progressbar.ProgressBar(max_value=progressbar.UnknownLength)
idx = 0

In [174]:
url = "https://www.data.gouv.fr/fr/datasets/r/97a18cbc-abe4-4a6d-8459-cdad7a174370"

In [175]:
# Va chercher
data = urllib.request.urlopen(url)

# display
idx += 1
bar.update(idx)

/ 0 Elapsed Time: 0:00:00                                                      

In [176]:
df = pd.read_csv(data, sep=';', dtype={'Bureau': str})

del df['Geo Shape']
del df['geo_point_2d']

# display
idx += 1
bar.update(idx)

- 2 Elapsed Time: 0:00:00                                                      

In [177]:
# Nombre de valeurs unique de codes bureau = nombre de lignes
assert df['Bureau'].nunique() == df.shape[0]

In [178]:
# Code bureau --> 3 caractères
assert (df['Bureau'].str.len() == 3).all()

In [179]:
# Colonne libellés --> non vides
assert not df['Libellés'].isnull().any()

In [180]:
# Colonne inscrits --> supérieur strict à 0
assert (df['Inscrits'] > 0).all()

In [181]:
# Colonne votants --> supérieur strict à 0
assert (df['NB_Votants'] > 0).all()

In [182]:
# Votants < Inscrits
assert (df['NB_Votants'] <= df['Inscrits']).all()

In [183]:
# Toutes les colonnes à partir de inscrits doivent être positives
assert (df.iloc[:, 2:] > 0).all().all()

In [184]:
# display
idx += 1
bar.update(idx)

\ 3 Elapsed Time: 0:00:00                                                      

In [185]:
result = True
for c in df.columns:
    if 'POURCENTAGE' in c:
        if not (df[c] <= 100).all():
            result = False

assert result

In [186]:
from sqlalchemy import create_engine

In [187]:
engine = create_engine("sqlite:///data.sqlite")
connection = engine.connect()

In [188]:
# Export des noms de bureaux
df[['Bureau', 'Libellés']].to_sql('bureaux', connection,
                                  if_exists='replace', index=False)

# display
idx += 1
bar.update(idx)

| 4 Elapsed Time: 0:00:00                                                      

In [189]:
# On crée la liste des colonnes contenant des nombres de votes
ll = ['Inscrits']

for c in df.columns:
    if 'NB_' in c:
        ll.append(c)

In [190]:
# Transformation : création de la table votes
pd.melt(
    df,
    id_vars=['Bureau'],
    value_vars=ll,
    var_name='Type de vote',
    value_name='Nombre de votes'
).to_sql('votes', connection, if_exists='replace', index=False)

# display
idx += 1
bar.update(idx)

/ 5 Elapsed Time: 0:00:01                                                      

In [191]:
# On crée la liste des colonnes contenant des pourcentags
ll = []

for c in df.columns:
    if 'POURCENTAGE_' in c:
        ll.append(c)

In [192]:
# Transformation : création de la table votes
pd.melt(
    df,
    id_vars=['Bureau'],
    value_vars=ll,
    var_name='Type de vote',
    value_name='Pourcentage'
).to_sql('pourcentages', connection, if_exists='replace', index=False)

# display
idx += 1
bar.update(idx)

- 6 Elapsed Time: 0:00:01                                                      

In [193]:
# Export des données brutes
df.to_sql('presidentielle', connection, if_exists='replace', index=False)

# display
idx += 1
bar.update(idx)

\ 7 Elapsed Time: 0:00:01                                                      

In [194]:
connection.close()

In [195]:
bar.finish()

| 7 Elapsed Time: 0:00:02                                                      
